In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from function import split_data
from sklearn.preprocessing import StandardScaler

# 检查是否有可用的 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
# 读取数据
data = pd.read_csv("../../data/dataset_reduced.csv")
X_train, X_test, y_train, y_test = split_data(data, 'Cs')

# 数据标准化
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 将y转换为 NumPy 数组
y_train = np.array(y_train)
y_test = np.array(y_test)

# 将数据转换为 PyTorch 张量并移动到 GPU
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1).to(device)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

print(X_train_tensor.shape)

torch.Size([499, 9])


In [8]:
# 创建 DataLoader
batch_size = 20
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [9]:
from torch_function import MAPE_Loss, RMSE_Loss
# 定义 ANN 模型
class ANN(nn.Module):
    def __init__(self, input_size):
        super(ANN, self).__init__()
        self.layer1 = nn.Linear(input_size, 120)
        self.layer2 = nn.Linear(120, 40)
        self.layer3 = nn.Linear(40, 50)
        self.layer4 = nn.Linear(50, 120)
        self.output = nn.Linear(120, 1)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = torch.relu(self.layer3(x))
        x = torch.relu(self.layer4(x))
        x = self.output(x)
        return x

# 初始化模型并将其移到 GPU（如果可用）
input_size = X_train.shape[1]
model = ANN(input_size).to(device)

# 定义损失函数和优化器
loss_function = RMSE_Loss().to(device)  # 自定义损失函数也要移到 GPU
optimizer = optim.Adam(model.parameters(), lr=0.002)

In [10]:
# 训练模型+包含早停
num_epochs = 3000
patience = 100  # 允许的最大连续未改进 epoch 数
best_loss = float('inf')  # 初始时验证损失设置为正无穷
cumulative_loss = 0.0
epochs_without_improvement = 0  # 连续未改进的 epoch 数

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        # 移动 batch 数据到 GPU
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = loss_function(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # 打印每10个 epoch 的损失
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

    # 计算验证损失
    model.eval()
    with torch.no_grad():
        # 在 GPU 上进行预测
        y_val_pred = model(X_test_tensor).to(device)
        # 验证损失计算时，确保 y_test_tensor 也在同一个设备上
        y_test_tensor = y_test_tensor.to(device)
        val_loss = loss_function(y_val_pred, y_test_tensor).item()  # 计算验证损失

    # 判断验证损失是否改善
    if val_loss < best_loss:
        best_loss = val_loss
        epochs_without_improvement = 0  # 重置计数器
        # 保存最佳模型
        torch.save(model.state_dict(), "ann_best_model.pth")
    else:
        epochs_without_improvement += 1

    # 如果验证损失在一定次数的 epoch 内没有改进，则停止训练
    if epochs_without_improvement >= patience:
        print(f"Early stopping at epoch {epoch + 1}")
        break

Epoch [10/3000], Loss: 26.0378
Epoch [20/3000], Loss: 19.9485
Epoch [30/3000], Loss: 16.7138
Epoch [40/3000], Loss: 14.8944
Epoch [50/3000], Loss: 13.3540
Epoch [60/3000], Loss: 13.8772
Epoch [70/3000], Loss: 12.3471
Epoch [80/3000], Loss: 11.2040
Epoch [90/3000], Loss: 10.5021
Epoch [100/3000], Loss: 9.5975
Epoch [110/3000], Loss: 9.7236
Epoch [120/3000], Loss: 9.4102
Epoch [130/3000], Loss: 8.9872
Epoch [140/3000], Loss: 8.8846
Epoch [150/3000], Loss: 8.9298
Epoch [160/3000], Loss: 7.6720
Epoch [170/3000], Loss: 8.4584
Epoch [180/3000], Loss: 7.7604
Epoch [190/3000], Loss: 7.5271
Epoch [200/3000], Loss: 7.1974
Epoch [210/3000], Loss: 8.3038
Epoch [220/3000], Loss: 7.7278
Epoch [230/3000], Loss: 7.9586
Epoch [240/3000], Loss: 7.6923
Epoch [250/3000], Loss: 8.0453
Epoch [260/3000], Loss: 6.9743
Epoch [270/3000], Loss: 7.6170
Epoch [280/3000], Loss: 7.7426
Epoch [290/3000], Loss: 6.7166
Epoch [300/3000], Loss: 6.5566
Epoch [310/3000], Loss: 7.4167
Epoch [320/3000], Loss: 7.2977
Epoch [3

In [11]:
from function import metrics_to_dataframe
# 加载模型参数，确保使用 weights_only=True 来提高安全性
model.load_state_dict(torch.load("ann_best_model.pth", weights_only=True))  # 加载模型参数

# 评估模型 (确保在 GPU 上)
model.eval()
model.to(device)

with torch.no_grad():
    y_train_pred = model(X_train_tensor).cpu().numpy()  # 转换为 CPU 数据，便于后续处理
    y_test_pred = model(X_test_tensor).cpu().numpy()

# 计算并显示评估指标
ann_metrics = metrics_to_dataframe(y_train, y_train_pred, y_test, y_test_pred, 'ANN')
ann_metrics

,model,R2_train,MAE_train,MAPE_train,RMSE_train,R2_test,MAE_test,MAPE_test,RMSE_test
0,ANN,0.967387,4.214927,4.424806,7.244716,0.962608,5.324582,7.202797,8.02384


In [7]:
ann_train = pd.DataFrame({'Actual': y_train, 'Predicted': y_train_pred.squeeze()})
ann_test = pd.DataFrame({'Actual': y_test, 'Predicted': y_test_pred.squeeze()})
ann_train.to_csv('ann_train.csv', index=False)
ann_test.to_csv('ann_test.csv', index=False)